In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats

def download_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, end=end_date)
    return data, stock.info

def calculate_standard_deviation(data):
    return data['Close'].pct_change().std() * np.sqrt(252)

def calculate_var(data, confidence_level=0.95):
    returns = data['Close'].pct_change().dropna()
    return np.percentile(returns, 100 * (1 - confidence_level))

def calculate_sharpe_ratio(data, risk_free_rate=0.02):
    returns = data['Close'].pct_change().dropna()
    excess_returns = returns - risk_free_rate / 252
    return np.sqrt(252) * excess_returns.mean() / excess_returns.std()

def calculate_sortino_ratio(data, risk_free_rate=0.02, target_return=0):
    returns = data['Close'].pct_change().dropna()
    downside_returns = returns[returns < target_return]
    excess_returns = returns - risk_free_rate / 252
    return np.sqrt(252) * excess_returns.mean() / downside_returns.std()

def calculate_max_drawdown(data):
    returns = data['Close'].pct_change().dropna()
    cumulative_returns = (1 + returns).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns / peak) - 1
    return drawdown.min()

def calculate_downside_deviation(data, target_return=0):
    returns = data['Close'].pct_change().dropna()
    downside_returns = returns[returns < target_return]
    return np.sqrt(np.mean(downside_returns**2))

def calculate_tracking_error(company_data, sector_index_data):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    return np.std(company_returns - index_returns) * np.sqrt(252)

def calculate_r_squared(company_data, sector_index_data):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    slope, intercept, r_value, _, _ = stats.linregress(index_returns, company_returns)
    return r_value**2

def calculate_treynor_ratio(company_data, beta, risk_free_rate=0.02):
    returns = company_data['Close'].pct_change().dropna()
    excess_returns = returns.mean() - risk_free_rate / 252
    return excess_returns / beta

def calculate_information_ratio(company_data, sector_index_data):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    return (company_returns.mean() - index_returns.mean()) / calculate_tracking_error(company_data, sector_index_data)

def calculate_cvar(data, confidence_level=0.95):
    returns = data['Close'].pct_change().dropna()
    var = calculate_var(data, confidence_level)
    return returns[returns <= var].mean()

def calculate_beta_adjusted_sharpe_ratio(company_data, sector_index_data, risk_free_rate=0.02):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    beta = stats.linregress(index_returns, company_returns).slope
    excess_returns = company_returns - risk_free_rate / 252
    return np.sqrt(252) * excess_returns.mean() / (excess_returns.std() * beta)

def calculate_drawdown_duration(data):
    returns = data['Close'].pct_change().dropna()
    cumulative_returns = (1 + returns).cumprod()
    drawdown = (cumulative_returns / cumulative_returns.cummax()) - 1
    drawdown_periods = (drawdown < 0).astype(int)
    return drawdown_periods.groupby((drawdown_periods != drawdown_periods.shift()).cumsum()).sum().max()

def calculate_ulcer_index(data):
    returns = data['Close'].pct_change().dropna()
    cumulative_returns = (1 + returns).cumprod()
    drawdown = (cumulative_returns / cumulative_returns.cummax()) - 1
    return np.sqrt(np.mean(drawdown**2))

def calculate_jensens_alpha(company_data, sector_index_data, risk_free_rate=0.02):
    company_returns = company_data['Close'].pct_change().dropna()
    index_returns = sector_index_data['Close'].pct_change().dropna()
    beta = stats.linregress(index_returns, company_returns).slope
    expected_return = risk_free_rate + beta * (index_returns.mean() * 252 - risk_free_rate)
    return company_returns.mean() * 252 - expected_return

def main():
    ticker = "NVDA"  # Replace with your desired stock ticker
    sector_index_ticker = "XLK"  # Replace with the appropriate sector index ticker
    start_date = "2012-01-01"
    end_date = "2024-06-30"

    company_data, company_info = download_stock_data(ticker, start_date, end_date)
    sector_index_data, _ = download_stock_data(sector_index_ticker, start_date, end_date)

    metrics = {
        'P/E Ratio': company_info.get('forwardEps') / company_info.get('previousClose') if company_info.get('forwardEps') and company_info.get('previousClose') else None,
        'P/B Ratio': company_info.get('priceToBook'),
        'Dividend Yield': company_info.get('dividendYield'),
        'Dividend Payout Ratio': company_info.get('payoutRatio'),
        'ROE': company_info.get('returnOnEquity'),
        'ROA': company_info.get('returnOnAssets'),
        'Beta': company_info.get('beta'),
        'Market Capitalization': company_info.get('marketCap'),
        'Revenue Growth': company_info.get('revenueGrowth'),
        'Debt-to-Equity Ratio': company_info.get('debtToEquity'),
        'Free Cash Flow': company_info.get('freeCashflow'),
        'Current Ratio': company_info.get('currentRatio'),
        'Quick Ratio': company_info.get('quickRatio'),
        'PEG Ratio': company_info.get('pegRatio'),
        'Standard Deviation': calculate_standard_deviation(company_data),
        'Value at Risk (VaR)': calculate_var(company_data),
        'Sharpe Ratio': calculate_sharpe_ratio(company_data),
        'Sortino Ratio': calculate_sortino_ratio(company_data),
        'Maximum Drawdown': calculate_max_drawdown(company_data),
        'Downside Deviation': calculate_downside_deviation(company_data),
        'Tracking Error': calculate_tracking_error(company_data, sector_index_data),
        'R-squared': calculate_r_squared(company_data, sector_index_data),
        'Treynor Ratio': calculate_treynor_ratio(company_data, beta=company_info.get('beta')),
        'Information Ratio': calculate_information_ratio(company_data, sector_index_data),
        'Conditional Value at Risk (CVaR)': calculate_cvar(company_data),
        'Beta-adjusted Sharpe Ratio': calculate_beta_adjusted_sharpe_ratio(company_data, sector_index_data),
        'Drawdown Duration': calculate_drawdown_duration(company_data),
        'Ulcer Index': calculate_ulcer_index(company_data),
        'Jensens Alpha': calculate_jensens_alpha(company_data, sector_index_data)
    }

    # Add metrics to the company data
    for metric, value in metrics.items():
        company_data[metric] = value

    # Save the data to a CSV file
    output_filename = f"{ticker}_data_with_metrics.csv"
    company_data.to_csv(output_filename)
    print(f"Data saved to {output_filename}")

if __name__ == "__main__":
    main()

Data saved to NVDA_data_with_metrics.csv
